In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
label_split = train.labels.apply(lambda x: x.split()) #chia 1 chuỗi các nhãn thành nhiều nhãn nếu có dấu cách
trans_label = MultiLabelBinarizer().fit(label_split)
labels = pd.DataFrame(trans_label.transform(label_split), columns=trans_label.classes_)

In [ ]:
labels.head()

In [ ]:
for label in labels.columns:
    print(labels[label].value_counts(normalize=True))

In [ ]:
submissions = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submissions.head()

In [ ]:
h_target = 224
w_target = 224
batch_size = 16

In [ ]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_dataframe(
    submissions,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    y_col=None,
    target_size=(h_target, w_target),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    shuffle=False,
    batch_size=batch_size
)

In [ ]:
model = keras.models.load_model("../input/resnet50-sub/ResNet50.h5")
preds = model.predict(test_generator)
print(preds)

In [ ]:
thresh = {
    'complex':0.33,
    'frog_eye_leaf_spot':0.33,
    'healthy':0.33,
    'powdery_mildew':0.33,
    'rust':0.33,
    'scab':0.33    
}

In [ ]:
for i in range(len(submissions['image'])):
    if preds[i][2] == np.max(preds[i]):
        submissions['labels'][i] == 'healthy'
    else:
        label_comb = []
        for j, label in enumerate(thresh.keys()):
            if preds[i][j] > thresh[label]:
                label_comb.append(label)
        submissions['labels'][i] = ' '.join([str(elem) for elem in label_comb]
  )
        if submissions['labels'][i] == ''or 'healthy' in submissions['labels'][i]:
            submissions['labels'][i] = ' '.join(labels.columns[:][preds[i] >= np.max(preds[i])])
submissions.to_csv('submission.csv', index=False) 

In [ ]:
submissions